<a href="https://colab.research.google.com/github/yantvrs/Data_structure_2/blob/main/ProjetoFinal/codes/Projeto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import pandas as pd
import osmnx as ox

ox.settings.log_console = True
ox.__version__

'1.8.0'

In [ ]:
def get_cycleways_graph(query):
    # Configuração das tags úteis para a biblioteca OSMnx
    useful_tags = ox.settings.useful_tags_way + ['cycleway'] + ['bicycle']
    ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)

    # Baixa o grafo da rede de ciclovias
    G = ox.graph_from_place(query=query, network_type='bike', simplify=False, retain_all=True)

    # Lista para armazenar as arestas que não são ciclovias
    non_cyc = []

    # Itera sobre as arestas do grafo
    for u, v, k, d in G.edges(keys=True, data=True):
        bi = False
        if "bicycle" in d and d['bicycle'] == 'designated':
            bi = True

        if d['highway'] != 'cycleway' and 'cycleway' not in d and not bi:
            non_cyc.append((u, v, k))

    # Remove as arestas que não são ciclovias
    G.remove_edges_from(non_cyc)

    # Remove nós isolados do grafo
    G = ox.utils_graph.remove_isolated_nodes(G)

    # Remove os nós que não são utilizados nas arestas
    unused_nodes = [node for node, degree in dict(G.degree()).items() if degree == 0]
    G.remove_nodes_from(unused_nodes)

    # Simplifica o grafo
    G = ox.simplify_graph(G)

    return G

In [ ]:
place1 = "São Paulo, Brazil"
G1 = get_cycleways_graph(place1)

<ipython-input-3-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


In [ ]:
place2 = "Amsterdam, Netherlands"
G2 = get_cycleways_graph(place2)

<ipython-input-3-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


In [ ]:
place3 = "Berlin, Germany"
G3 = get_cycleways_graph(place3)

<ipython-input-3-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


#Análise Comparativa

##Density Based Stats

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> SÃO PAULO <<<<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place1)
areaSP = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G1, area=areaSP)
pd.Series(stats)

n                                                                            1509
m                                                                            1881
k_avg                                                                    2.493042
edge_length_total                                                      726266.943
edge_length_avg                                                        386.106828
streets_per_node_avg                                                     3.056329
streets_per_node_counts         {0: 0, 1: 136, 2: 73, 3: 943, 4: 287, 5: 68, 6...
streets_per_node_proportions    {0: 0.0, 1: 0.09012591119946985, 2: 0.04837640...
intersection_count                                                           1373
street_length_total                                                    441314.035
street_segment_count                                                         1194
street_length_avg                                                      369.609745
circuity_avg    

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> AMSTERDAM <<<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place2)
areaAMS = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G2, area=areaAMS)
pd.Series(stats)

n                                                                            6307
m                                                                           11454
k_avg                                                                    3.632155
edge_length_total                                                     1406328.214
edge_length_avg                                                        122.780532
streets_per_node_avg                                                     3.152846
streets_per_node_counts         {0: 0, 1: 167, 2: 544, 3: 3923, 4: 1539, 5: 10...
streets_per_node_proportions    {0: 0.0, 1: 0.026478515934675758, 2: 0.0862533...
intersection_count                                                           6140
street_length_total                                                    871651.738
street_segment_count                                                         7240
street_length_avg                                                      120.393886
circuity_avg    

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> BERLIN <<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place3)
areaBER = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G3, area=areaBER)
pd.Series(stats)

n                                                                            6437
m                                                                            8087
k_avg                                                                    2.512661
edge_length_total                                                     1650591.237
edge_length_avg                                                        204.104271
streets_per_node_avg                                                      2.93149
streets_per_node_counts         {0: 0, 1: 804, 2: 905, 3: 2955, 4: 1511, 5: 23...
streets_per_node_proportions    {0: 0.0, 1: 0.12490290508000622, 2: 0.14059344...
intersection_count                                                           5633
street_length_total                                                   1063514.305
street_segment_count                                                         5555
street_length_avg                                                       191.45172
circuity_avg    

## Outras métricas

In [ ]:
#CICLOVIA POR HABITANTE

populacaoSP = 11451245
populacaoAMS = 1603531
populacaoBER = 3711930

#Armazenando o comprimento (em metros) de ciclovias [retirados da função stats]
comprimento_ciclovias_SP = ox.stats.street_length_total(ox.utils_graph.get_undirected(G1))
comprimento_ciclovias_AMS = ox.stats.street_length_total(ox.utils_graph.get_undirected(G2))
comprimento_ciclovias_BER = ox.stats.street_length_total(ox.utils_graph.get_undirected(G3))


ciclovias_por_habitanteSP = comprimento_ciclovias_SP / populacaoSP
ciclovias_por_habitanteAMS = comprimento_ciclovias_AMS / populacaoAMS
ciclovias_por_habitanteBER = comprimento_ciclovias_BER / populacaoBER

print(f"Metros de ciclovias por habitante em São Paulo: {ciclovias_por_habitanteSP}")
print(f"Metros de ciclovias por habitante em Amsterdã: {ciclovias_por_habitanteAMS}")
print(f"Metros de ciclovias por habitante em Berlim: {ciclovias_por_habitanteBER}")



Metros de ciclovias por habitante em São Paulo: 0.0385385200473835
Metros de ciclovias por habitante em Amsterdã: 0.5435827171411101
Metros de ciclovias por habitante em Berlim: 0.2865124894596613


In [ ]:
# DENSIDADE DE CICLOVIAS

# Calcula a densidade de ciclovias
densidade_ciclovias_SP = comprimento_ciclovias_SP / areaSP * 1000000
print(f"Densidade de ciclovias em São Paulo: {densidade_ciclovias_SP} metros por quilômetro quadrado")

densidade_ciclovias_AMS = comprimento_ciclovias_AMS / areaAMS * 1000000
print(f"Densidade de ciclovias em Amesterdã: {densidade_ciclovias_AMS} metros por quilômetro quadrado")

densidade_ciclovias_BER = comprimento_ciclovias_BER / areaBER * 1000000
print(f"Densidade de ciclovias Berlim: {densidade_ciclovias_BER} metros por quilômetro quadrado")

Densidade de ciclovias em São Paulo: 289.85625981309266 metros por quilômetro quadrado
Densidade de ciclovias em Amesterdã: 3972.5555610337083 metros por quilômetro quadrado
Densidade de ciclovias Berlim: 1194.033704511825 metros por quilômetro quadrado
